In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn import model_selection
from sklearn.decomposition import PCA 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report, plot_roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import GaussianNB 
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering


warnings.filterwarnings("ignore")
%matplotlib inline
sns.set(rc={'figure.figsize':(11,10)})
sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)

In [ ]:
# Funcion auxiliar
def make_confusion_matriz(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.

    Arguments
    ---------
    cf:            confusion matrix to be passed in

    group_names:   List of strings that represent the labels row by row to be shown in each square.

    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'

    count:         If True, show the raw number in the confusion matrix. Default is True.

    normalize:     If True, show the proportions for each category. Default is True.

    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.

    xyticks:       If True, show x and y ticks. Default is True.

    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.

    sum_stats:     If True, display summary statistics below the figure. Default is True.

    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.

    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.

    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('Valores Reales')
        plt.xlabel('Valores Predichos' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
df=pd.read_csv("mantenimiento.csv")

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
list(df.columns)

In [ ]:
# Necesitamos hacer
# 1 Descartar UDI y Product ID
# 2 Hacer numerica Type L=0, M=1, H=2
# 3 Hacer numerica Failure Type
# 4 Separar Features y Target como target1 y Failure Type como target2
# 5 Escalar el resto como features

In [ ]:
# Veamos los valores distintos de Failure Type
df['Failure Type'].unique()

In [ ]:
# Veamos los valores distintos de Failure Type
df['Type'].unique()

In [ ]:
# Veamos los valores distintos de Failure Type
df['Target'].unique()

In [ ]:
# 1 Descartar UDI y Product ID
df.drop(df.columns[:2],axis=1,inplace=True)

In [ ]:
# 2 Hacer numerica Type L=0, M=1, H=2
# 3 Hacer numerica Failure Type

dict_reemplazo = {"Type":     {"L": 0, "M": 1, 'H': 2},
                   "Failure Type":     {'No Failure' : 0,
                                        'Power Failure' : 1,
                                        'Tool Wear Failure' : 2,
                                        'Overstrain Failure' : 3,
                                        'Random Failures' : 4,
                                        'Heat Dissipation Failure' : 5}
               }
# Aplicamos el reemplazo en el dataframe
df = df.replace(dict_reemplazo)
df.sample(10)

In [ ]:
# Veamos los valores distintos de Failure Type
df['Type'].unique()

In [ ]:
df.info()

In [ ]:
# 4 Separar Features y Target como target1 y Failure Type como target2
X = df.iloc[:,0:-2]
y_tg1 = df.iloc[:,-2:-1]
y_tg2 = df.iloc[:,-1:]

In [ ]:
#sns.pairplot(df);

In [ ]:
sns.countplot(x='Target', data=df);

In [ ]:
sns.countplot(x='Failure Type', data=df);

In [ ]:
# contamos los valores distintos
df['Target'].value_counts()

In [ ]:
# contamos los valores distintos
df['Failure Type'].value_counts()

In [ ]:
df.Type.value_counts()

In [ ]:
df.describe()

Vamos a analizar con falla/no falla

In [ ]:
# Para escalar 
# Normalizamos todo el dataset
X_normal=(X - X.mean()) / X.std()
X_normal

In [ ]:
# Hacemos PCA
#Componentes principales
pca = PCA()
pca.fit(X_normal,y_tg1)
x_new = pca.transform(X_normal)

In [ ]:
# Varianza explicada por cada componente
var_pc = pca.explained_variance_ratio_
#print(var_pc)

indice = 1
for var in var_pc:
    print("PCA{} : {}".format(indice, round(var,4)))
    indice += 1

In [ ]:
# Seleccionamos los nombres de las columnas
nombres = list(X_normal.columns)

In [ ]:
from matplotlib import colors
import matplotlib.patches as mpatches

green_patch = mpatches.Patch(color='green', label='No Falla')
red_patch = mpatches.Patch(color='red', label='Falla')


# asignamos los dos primeros PC a los ejes coordinados
x_valor = x_new[:,0]
y_valor = x_new[:,1]

cmap = colors.ListedColormap(['green', 'red'])

dot = list(y_tg1.Target.values)

# hacemos un grafico de dispersion
dispersion_1 = plt.scatter(x_valor, y_valor , c=dot, cmap=cmap)

# Le ponemos nombre a los ejes
plt.xlabel("PC1");
plt.ylabel("PC2");

# mostramos la leyenda
plt.legend(handles=[green_patch, red_patch], fontsize=12)

# Mostamos la grilla
plt.grid();

# imprimimos el grafico completo
plt.show();

In [ ]:
# Definimos el rango de los ejes del grafico
plt.axis([-0.4,1,-0.75,0.75])

# Vemos cuantos vectores son las direcciones de maxima varianza
n = pca.components_.shape[0]

# Recorremos esos vectores y los vamos dibujando en el plano
for i in range(n):
    plt.arrow(0, 0, pca.components_[i,0], pca.components_[i,1], color = 'r', alpha = 1);
    # En el extremo de cada vector ponemos en nombre de la columan correspondiente (un poco dezplazados)
    plt.text(pca.components_[i,0]*1.1 , pca.components_[i,1]*1.1, nombres[i], color = 'g', ha = 'center', va = 'center', fontsize=12);

# Le ponemos nombre a los ejes
plt.xlabel("PC1");
plt.ylabel("PC2");

# imprimimos el grafico completo
plt.show();

Clasificacion Supervisada


In [ ]:
# Separamos en train/test
X_train, X_test, y_train, y_test = train_test_split(X_normal, y_tg1, stratify=y_tg1, test_size=0.2, random_state=50000)

In [ ]:
# Probamos con regresion logistica
lg = LogisticRegression() 
modelo_lg = lg.fit(X_train, y_train) 
y_pred_test = modelo_lg.predict(X_test) 

# matriz de confusión
conf = confusion_matrix(y_test,y_pred_test)

# grafico matriz de confusión
labels = ['VN','FP','FN','VP']
categories = ['No Falla', 'Falla']
make_confusion_matriz(conf, 
                     group_names=labels,
                     categories=categories, 
                     cmap='magma',
                     cbar=False,
                     sum_stats=False,
                     figsize=(6,6))


In [ ]:
# funciona solamente para clasificación binaria (2 clases)
# para muchas clases se puede ver esto: 
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py
plot_roc_curve(modelo_lg, X_test, y_test);

In [ ]:
# predigo probabilidades según el modelo con los datos de TESTEO.
# Luego uso estas probabilidades para, mediante punto de corte, clasificar en clases.
probas = modelo_lg.predict_proba(X_test)

# array con los puntos de corte
puntos_corte = np.arange(0.02, 1, 0.02) # empieza, termina, paso
acc = [] #array donde voy a ir guardando los valores de accuracy para cada punto de corte
recall = [] #ídem para recall
prec = [] #ídem para precision
f1 = [] #íðem para f1 score


for punto in puntos_corte: #para cada punto de corte

  # calculo las predicciones para ese punto de corte
  y_pred_test_custom = np.ones(y_test.shape)
  for i in range(probas.shape[0]):
      if (probas[i,0]>punto):
          y_pred_test_custom[i]=0.
  
  # calculo las métricas para ese punto de corte y las guardo los arrays antes creados
  acc.append( accuracy_score(y_test, y_pred_test_custom) ) #.append agrega al final del array
  recall.append( recall_score(y_test, y_pred_test_custom) )
  prec.append( precision_score(y_test, y_pred_test_custom) )
  f1.append( f1_score(y_test, y_pred_test_custom) )


# LO QUE SIGUE ES UN GRÁFICO de los valores guardados en los arrays: acc, recall, prec y f1.
# SE PUEDE trabajar directamente con los arrays para sacar más información.

plt.plot(puntos_corte, acc, puntos_corte, recall, puntos_corte, prec, puntos_corte, f1)
# plt.xlim(0,1)
# plt.ylim(0.1)
plt.xlabel('Punto de corte')
plt.yticks(np.arange(0,1.05,0.05))
plt.xticks(np.arange(0,1.05,0.05))
plt.grid()
plt.axvline(x=0.95, label='Punto de corte', ls=':', lw=2, c='gray')
plt.legend(['Accuracy','Recall','Precision','F1 score','Punto de corte']);

In [ ]:
probas = lg.predict_proba(X_test) #probabilidades de cada clase según modelo predictivo
# inspecciono esas probabilidades
with np.printoptions(precision=3, suppress=True):
    print(probas[0:10])

# por ejemplo, por defecto asumo que no falla (0)
y_pred_custom = np.zeros(y_test.shape)

#y si la probabilidad de y=1 es mayor a 0.85, lo clasifico como falla (1)
for i in range(probas.shape[0]):
    if (probas[i,0]<0.95):
        y_pred_custom[i]=1.


conf_custom = confusion_matrix(y_test,y_pred_custom)

# grafico matriz de confusión
labels = ['VN','FP','FN','VP']
categories = ['No Falla', 'Falla']
make_confusion_matriz(conf_custom, 
                     group_names=labels,
                     categories=categories, 
                     cmap='magma',
                     cbar=False,
                     sum_stats=False,
                     figsize=(6,6))


In [ ]:
# Analizamos con Bayes Ingenuo
# instanciamos el modelo Bayes ingenuo
gnb = GaussianNB() 

# ajustamos nuestros features y target
modelo_gnb = gnb.fit(X_train, y_train) 

# Ahora hacemos que prediga con nuestros features
y_nb_pred = modelo_gnb.predict(X_test) 

# matriz de confusión
conf_nb = confusion_matrix(y_test,y_nb_pred)

# grafico matriz de confusión
labels = ['VN','FP','FN','VP']
categories = ['No Falla', 'Falla']
make_confusion_matriz(conf_nb, 
                     group_names=labels,
                     categories=categories, 
                     cmap='magma',
                     cbar=False,
                     figsize=(6,6))



In [ ]:
# funciona solamente para clasificación binaria (2 clases)
# para muchas clases se puede ver esto: 
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py
plot_roc_curve(modelo_gnb, X_test, y_test);

In [ ]:
# predigo probabilidades según el modelo con los datos de TESTEO.
# Luego uso estas probabilidades para, mediante punto de corte, clasificar en clases.
probas = modelo_gnb.predict_proba(X_test)

# array con los puntos de corte
puntos_corte = np.arange(0.02, 1, 0.02) # empieza, termina, paso
acc = [] #array donde voy a ir guardando los valores de accuracy para cada punto de corte
recall = [] #ídem para recall
prec = [] #ídem para precision
f1 = [] #íðem para f1 score


for punto in puntos_corte: #para cada punto de corte

  # calculo las predicciones para ese punto de corte
  y_pred_test_custom = np.ones(y_test.shape)
  for i in range(probas.shape[0]):
      if (probas[i,0]>punto):
          y_pred_test_custom[i]=0.
  
  # calculo las métricas para ese punto de corte y las guardo los arrays antes creados
  acc.append( accuracy_score(y_test, y_pred_test_custom) ) #.append agrega al final del array
  recall.append( recall_score(y_test, y_pred_test_custom) )
  prec.append( precision_score(y_test, y_pred_test_custom) )
  f1.append( f1_score(y_test, y_pred_test_custom) )


# LO QUE SIGUE ES UN GRÁFICO de los valores guardados en los arrays: acc, recall, prec y f1.
# SE PUEDE trabajar directamente con los arrays para sacar más información.

plt.plot(puntos_corte, acc, puntos_corte, recall, puntos_corte, prec, puntos_corte, f1)
# plt.xlim(0,1)
# plt.ylim(0.1)
plt.xlabel('Punto de corte')
plt.yticks(np.arange(0,1.05,0.05))
plt.xticks(np.arange(0,1.05,0.05))
plt.grid()
plt.axvline(x=0.875, label='Punto de corte', ls=':', lw=2, c='gray')
plt.legend(['Accuracy','Recall','Precision','F1 score','Punto de corte']);

In [ ]:
probas = gnb.predict_proba(X_test) #probabilidades de cada clase según modelo predictivo
# inspecciono esas probabilidades
with np.printoptions(precision=3, suppress=True):
    print(probas[0:10])

# por ejemplo, por defecto asumo que no falla (0)
y_pred_custom = np.zeros(y_test.shape)

#y si la probabilidad de y=1 es mayor a 0.85, lo clasifico como falla (1)
for i in range(probas.shape[0]):
    if (probas[i,0]<0.875):
        y_pred_custom[i]=1.


conf_custom = confusion_matrix(y_test,y_pred_custom)

# grafico matriz de confusión
labels = ['VN','FP','FN','VP']
categories = ['No Falla', 'Falla']
make_confusion_matriz(conf_custom, 
                     group_names=labels,
                     categories=categories, 
                     cmap='magma',
                     cbar=False,
                     sum_stats=False,
                     figsize=(6,6))


In [ ]:
# Analizamos con Arboles de decision
# instanciamos el modelo 
# los parametros se ajustaron con prueba y error
dtc = tree.DecisionTreeClassifier(criterion='gini',
                                             min_samples_split=20,
                                             min_samples_leaf=5,
                                             max_depth = 5,
                                             class_weight={1:25})

# ajustamos nuestros features y target
modelo_dtc = dtc.fit(X_train, y_train) 

# Ahora hacemos que prediga con nuestros features
y_dt_pred = modelo_dtc.predict(X_test) 

# matriz de confusión
conf_dt = confusion_matrix(y_test,y_dt_pred)

# grafico matriz de confusión
labels = ['VN','FP','FN','VP']
categories = ['No Falla', 'Falla']
make_confusion_matriz(conf_dt, 
                     group_names=labels,
                     categories=categories, 
                     cmap='magma',
                     cbar=False,
                     sum_stats=False,
                     figsize=(6,6))


In [ ]:
# predigo probabilidades según el modelo con los datos de TESTEO.
# Luego uso estas probabilidades para, mediante punto de corte, clasificar en clases.
probas = modelo_dtc.predict_proba(X_test)

# array con los puntos de corte
puntos_corte = np.arange(0.02, 1, 0.02) # empieza, termina, paso
acc = [] #array donde voy a ir guardando los valores de accuracy para cada punto de corte
recall = [] #ídem para recall
prec = [] #ídem para precision
f1 = [] #íðem para f1 score


for punto in puntos_corte: #para cada punto de corte

  # calculo las predicciones para ese punto de corte
  y_pred_test_custom = np.ones(y_test.shape)
  for i in range(probas.shape[0]):
      if (probas[i,0]>punto):
          y_pred_test_custom[i]=0.
  
  # calculo las métricas para ese punto de corte y las guardo los arrays antes creados
  acc.append( accuracy_score(y_test, y_pred_test_custom) ) #.append agrega al final del array
  recall.append( recall_score(y_test, y_pred_test_custom) )
  prec.append( precision_score(y_test, y_pred_test_custom) )
  f1.append( f1_score(y_test, y_pred_test_custom) )


# LO QUE SIGUE ES UN GRÁFICO de los valores guardados en los arrays: acc, recall, prec y f1.
# SE PUEDE trabajar directamente con los arrays para sacar más información.

plt.plot(puntos_corte, acc, puntos_corte, recall, puntos_corte, prec, puntos_corte, f1)
# plt.xlim(0,1)
# plt.ylim(0.1)
plt.xlabel('Punto de corte')
plt.yticks(np.arange(0,1.05,0.05))
plt.xticks(np.arange(0,1.05,0.05))
plt.grid()
plt.axvline(x=0.5, label='Punto de corte', ls=':', lw=2, c='gray')
plt.legend(['Accuracy','Recall','Precision','F1 score','Punto de corte']);

In [ ]:
probas = dtc.predict_proba(X_test) #probabilidades de cada clase según modelo predictivo
# inspecciono esas probabilidades
with np.printoptions(precision=3, suppress=True):
    print(probas[0:10])

# por ejemplo, por defecto asumo que no falla (0)
y_pred_custom = np.zeros(y_test.shape)

#y si la probabilidad de y=1 es mayor a 0.85, lo clasifico como falla (1)
for i in range(probas.shape[0]):
    if (probas[i,0]<0.4):
        y_pred_custom[i]=1.


conf_custom = confusion_matrix(y_test,y_pred_custom)

# grafico matriz de confusión
labels = ['VN','FP','FN','VP']
categories = ['No Falla', 'Falla']
make_confusion_matriz(conf_custom, 
                     group_names=labels,
                     categories=categories, 
                     cmap='magma',
                     cbar=False,
                     sum_stats=False, 
                     figsize=(6,6))

In [ ]:
# funciona solamente para clasificación binaria (2 clases)
# para muchas clases se puede ver esto: 
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py
plot_roc_curve(modelo_dtc, X_test, y_test);

In [ ]:
# Hacemos PCA
#Componentes principales
pca = PCA()
pca.fit(X_normal,y_tg1)
x_new = pca.transform(X_normal)

In [ ]:
# instancio la clase
pca = PCA(n_components=2)
# calculo PCA
pca.fit(X_normal)
# transformo los datos originales al plano PCA
pca_data = pca.transform(X_normal)

# cálculo varianza explicada
vars = pca.explained_variance_ratio_
var1 = round(100*vars[0],2)
var2 = round(100*vars[1],2)

# gráfico
plt.scatter(x=pca_data[:,0], y=pca_data[:,1], lw=2)
plt.xlabel("Componente principal 1 ({}%)".format(var1))
plt.ylabel("Componente principal 2 ({}%)".format(var2))
plt.title("Análisis por componentes principales")
plt.show()

In [ ]:
silhouette_scores = []
rango_clusters = range(2,6)

for i in rango_clusters:
    # para cada valor de i, calculo kmeans y silhouette
    k_means = KMeans(n_clusters=i,random_state=100000, n_init=20)
    #k_means = KMeans(n_clusters=i)
    k_means.fit(X_normal)
    
    sil_score = silhouette_score(X_normal, labels=k_means.labels_)
    silhouette_scores.append(sil_score)

plt.plot(rango_clusters,silhouette_scores)
plt.xticks(rango_clusters)
plt.xlabel('Cantidad de clusters')
plt.ylabel('Score Silhouette')
plt.title('Los máximos locales son los candidatos según Silhouette')
plt.show()

Clasificacion no Supervisada


Ahora con Aglomerativo


In [ ]:
politica_agrupamiento = ['ward', 'complete', 'average', 'single']
cant_cluster = 2

In [ ]:
clustering = AgglomerativeClustering(n_clusters=cant_cluster, linkage=politica_agrupamiento[0])

clustering.fit(X_normal)

y_jerarq = pd.Series(clustering.labels_)

# grafico en el plano PCA datos y clusters
plt.scatter(pca_data[:,0],pca_data[:,1], c=clustering.labels_, cmap="Paired")
plt.title("Visualización de clusters por PCA")
plt.show()

In [ ]:
# Agrega una nueva variable con el cluster al que pertenece cada observación
df["Cluster"] = y_jerarq

# Calculo la media de cada variable para cada cluster
df.groupby(["Cluster"]).mean()

In [ ]:
# Calculo la desviacion de cada variable para cada cluster
df.groupby(["Cluster"]).std()

In [ ]:
df.pop('Cluster')

In [ ]:
clustering = AgglomerativeClustering(n_clusters=cant_cluster, linkage=politica_agrupamiento[1])

clustering.fit(X_normal)

y_jerarq = pd.Series(clustering.labels_)

# grafico en el plano PCA datos y clusters
plt.scatter(pca_data[:,0],pca_data[:,1], c=clustering.labels_, cmap="Paired")
plt.title("Visualización de clusters por PCA")
plt.show()

In [ ]:
# Agrega una nueva variable con el cluster al que pertenece cada observación
df["Cluster"] = y_jerarq

# Calculo la media de cada variable para cada cluster
df.groupby(["Cluster"]).mean()

In [ ]:
# Calculo la desviacion de cada variable para cada cluster
df.groupby(["Cluster"]).std()

In [ ]:
df.pop('Cluster')

In [ ]:
clustering = AgglomerativeClustering(n_clusters=cant_cluster, linkage=politica_agrupamiento[2])

clustering.fit(X_normal)

y_jerarq = pd.Series(clustering.labels_)

# grafico en el plano PCA datos y clusters
plt.scatter(pca_data[:,0],pca_data[:,1], c=clustering.labels_, cmap="Paired")
plt.title("Visualización de clusters por PCA")
plt.show()

In [ ]:
# Agrega una nueva variable con el cluster al que pertenece cada observación
df["Cluster"] = y_jerarq

# Calculo la media de cada variable para cada cluster
df.groupby(["Cluster"]).mean()

In [ ]:
# Calculo la desviacion de cada variable para cada cluster
df.groupby(["Cluster"]).std()

In [ ]:
df.pop('Cluster')

In [ ]:
clustering = AgglomerativeClustering(n_clusters=cant_cluster, linkage=politica_agrupamiento[3])

clustering.fit(X_normal)

y_jerarq = pd.Series(clustering.labels_)

# grafico en el plano PCA datos y clusters
plt.scatter(pca_data[:,0],pca_data[:,1], c=clustering.labels_, cmap="Paired")
plt.title("Visualización de clusters por PCA")
plt.show()

In [ ]:
# Agrega una nueva variable con el cluster al que pertenece cada observación
df["Cluster"] = y_jerarq

# Calculo la media de cada variable para cada cluster
df.groupby(["Cluster"]).mean()

In [ ]:
# Calculo la desviacion de cada variable para cada cluster
df.groupby(["Cluster"]).std()

In [ ]:
df.pop('Cluster')

Lo anterior se podria haber resuelto con una funcion que tome como parametro el linkage pero por las limitaciones de graficacion de notebbok se decidio copiar y pegar